## MONTE CARLO SIMULATION 

**Key idea**: simulate many joint paths of (S_t, v_t) under the Heston model using the QE scheme, compute the option payoff at maturity for each path, and estimate the option price as the discounted average of these payoffs.

In [ ]:
import numpy as np


S0      = 105.0 # spot price
K      = 145   # strike price
T      = 1.5   # time to maturity (years)
r      = 0.09  # risk-free rate
q      = 0.0   # dividend yield
v0     = 0.075 # initial variance
kappa  = 4.0   # speed of mean reversion
theta  = 0.05  # long-term variance
sigma_v= 0.4   # vol-of-vol
rho    = -0.3  # correlation

steps=200
paths=100_000
antithetic=True # simulate opposite shocks (eg. t: to high; t+1: to low)
seed=None
option_type="call"

**QE STEP**: Given the current variance $v_t$, the goal is to simulate $v_{t+\Delta t}$ in the Heston model without ever producing negative values.
Instead of using an Euler discretization (which may yield negative variance), the QE scheme constructs a random variable $v_{t+\Delta}$ that:
- has the same conditional mean and variance as the CIR process,
- is always non-negative.

First, we define the conditional moments:
$$m = \mathbb{E}[v_{t+\Delta}\mid v_t],
\qquad
s^2 = \mathrm{Var}(v_{t+\Delta}\mid v_t)$$

The QE scheme aims to approximate the conditional distribution of the CIR process using only these two moments.

We then define $$\psi = \frac{s^2}{m^2}$$
which measures how “noisy” the variance is relative to its mean.
If $\psi$ is small, the distribution of $v_{t+\Delta}$ is tightly concentrated around $m$; \
if $\psi$ is large, a significant amount of probability mass is concentrated near zero.

Based on the value of $\psi$, the QE scheme distinguishes TWO cases (with threshold $\psi_c = 1.5$):
- Small $\psi$: $v_{t+\Delta}$ is generated as the square of a shifted normal random variable, with parameters chosen to match the conditional mean and variance.
- Large $\psi$: $v_{t+\Delta}$ is generated from a mixture distribution (point mass at zero plus an exponential component) to accurately capture the probability mass near zero.

In [2]:
def _qe_step(v_curr, Z1, U, theta, kappa, sigma_v, ed):
    """
    QE scheme for variance evolution (Andersen)
    """
    # Conditional moments
    m = theta + (v_curr - theta) * ed # mean
    s2 = (v_curr * sigma_v**2 * ed * (1.0 - ed) / kappa +
          theta * sigma_v**2 * (1.0 - ed)**2 / (2.0 * kappa)) # variance
    
    # Numerical safeguard
    m = np.maximum(m, 1e-14)
    psi = s2 / (m**2)
    
    # Threshold
    psi_c = 1.5
    
    # CASE 1: Noncentral chi-square approximation (low psi)
    idx1 = psi <= psi_c
    sqrt_term = np.sqrt(2.0 / psi[idx1])
    b2 = 2.0 / psi[idx1] - 1.0 + sqrt_term * np.sqrt(sqrt_term**2 - 1.0)
    a = m[idx1] / (1.0 + b2)
    v_next_1 = a * (np.sqrt(b2) + Z1[idx1])**2
    
    # CASE 2: Exponential mixture (high psi)
    idx2 = ~idx1
    p = (psi[idx2] - 1.0) / (psi[idx2] + 1.0)
    p = np.clip(p, 0.0, 1.0 - 1e-14)
    beta = (1.0 - p) / m[idx2]
    v_next_2 = np.where(U[idx2] <= p, 0.0, 
                        -np.log((1.0 - p) / (1.0 - U[idx2])) / beta)
    
    # Combine cases
    v_next = np.empty_like(v_curr)
    v_next[idx1] = v_next_1
    v_next[idx2] = v_next_2
    v_next = np.maximum(v_next, 0.0)
    
    return v_next

**STOCK STEP**: The values $v_t$ and $v_{t+\Delta}$ are used to update the asset price $S_t$ in a way that is consistent with the Heston model and with the correlation parameter \rho.

From the integration of the SDE for the asset price, we have
$$\Delta \ln S \;\approx\; (r-q)\Delta t
\;-\;\frac12\int_t^{t+\Delta} v_s\,ds
\;+\;\int_t^{t+\Delta}\sqrt{v_s}\,dW_s^S$$

Therefore, we need to approximate the two integrals appearing in this expression.
- Variance integral
I_1 approximates the integrated variance using the trapezoidal rule:
$$I_1 \;\approx\; \int_t^{t+\Delta} v_s\,ds
\;\approx\; \frac{v_t + v_{t+\Delta}}{2}\,\Delta t$$
- Correlated stochastic integral
Using the variance dynamics, we approximate
$$\int_t^{t+\Delta}\sqrt{v_s}\,dW_s^v
\;\approx\; \frac{1}{\sigma_v}
\Big[(v_{t+\Delta}-v_t) - \kappa(\theta-v_t)\Delta t\Big]$$
Multiplying by $\rho$, we obtain the drift correction term
$$\text{drift\_corr}
\;\approx\;
\frac{\rho}{\sigma_v}
\Big[(v_{next}-v_{curr}) - \kappa(\theta-v_{curr})\Delta t\Big]$$

The price update is then given by the conditional lognormal discretization
$$S_{t+\Delta}
= S_t \exp\Big(
(r-q)\Delta t
-\frac12 I_1
+\text{drift\_corr}
+\sqrt{(1-\rho^2)I_1}\,Z_2
\Big)$$

Note that $Z_2$ represents only the independent component of the Brownian motion, according to the decomposition
$dW^S = \rho\,dW^v + \sqrt{1-\rho^2}\,dW^\perp$

In [3]:
def _stock_step(S_curr, v_curr, v_next, Z2, r, q, rho, kappa, theta, sigma_v, dt):
    # pathwise estimation
    I1 = 0.5 * (v_curr + v_next) * dt
    #I1 = np.maximum(I1, 0.0)

    # Andersen QE II
    drift_corr = (rho / sigma_v) * ( (v_next - v_curr) - kappa * (theta - v_curr) * dt )

    S_next = S_curr * np.exp(
        (r - q) * dt
        - 0.5 * I1
        + drift_corr
        + np.sqrt(np.maximum((1.0 - rho**2) * I1, 0.0)) * Z2
    )
    return S_next

**STEPS**:
- Initialize the state variables $S$ and $v$ at the initial time $t=0$.
- For each time interval $(t, t+\Delta t)$:
	- generate the random variables;
	- compute $v_{t+\Delta t}$ from $v_t$ using the QE scheme;
	- compute $S_{t+\Delta t}$ from $S_t$, $v_t$, and $v_{t+\Delta t}$;
	- update the variance to the new value.
- At maturity, compute the payoff for each simulated path and obtain the option price as the discounted average of the payoffs (for a European call): $(S_T - K)^+ = \max(S_T - K, 0)$

The call price is then given by
$$\text{Option Price}
=
e^{-rT}\,\mathbb{E}^{\mathbb{Q}}[(S_T - K)^+]
\;\approx\;
e^{-rT}\,\frac{1}{N}\sum_{i=1}^{N}\max(S_T^{(i)} - K, 0)$$
where $N$ is the number of simulated paths.


In [ ]:
rng = np.random.default_rng(seed)
dt = T / steps
ed = np.exp(-kappa * dt)

# Adjust for antithetic variates
n = paths // 2 if antithetic else paths

# Initialize arrays
S = np.full(n, S0, dtype=np.float64)
v = np.full(n, v0, dtype=np.float64)

if antithetic:
    S_a = np.full(n, S0, dtype=np.float64)
    v_a = np.full(n, v0, dtype=np.float64)

# Time stepping; each iteration is an interval [t, t+dt]
for _ in range(steps):
    # Random variables
    Z1 = rng.standard_normal(n)
    Z2 = rng.standard_normal(n)
    U = rng.random(n)
    
    # === Normal paths ===
    v_next = _qe_step(v, Z1, U, theta, kappa, sigma_v, ed)
    S = _stock_step(S, v, v_next, Z2, r, q, rho, kappa, theta, sigma_v, dt)
    v = v_next
    
    # === Antithetic paths ===
    if antithetic:
        # Z1 --> -Z1
        # U --> 1 - U
        # Z2 --> -Z2
        v_next_a = _qe_step(v_a, -Z1, 1.0 - U, theta, kappa, sigma_v, ed)
        S_a = _stock_step(S_a, v_a, v_next_a, -Z2, r, q, rho, kappa, theta, sigma_v, dt)
        v_a = v_next_a


# Compute payoffs
disc = np.exp(-r * T)
if option_type.lower() == "call":
    payoff = np.maximum(S - K, 0.0)
    if antithetic:
        payoff_a = np.maximum(S_a - K, 0.0)
else:
    payoff = np.maximum(K - S, 0.0)
    if antithetic:
        payoff_a = np.maximum(K - S_a, 0.0)

# Combine antithetic paths
if antithetic:
    combined_payoff = 0.5 * (payoff + payoff_a)
    m_pay = np.mean(combined_payoff)
    std_pay = np.std(combined_payoff, ddof=1)
    n_eff = n   
else:
    m_pay = np.mean(payoff)
    std_pay = np.std(payoff, ddof=1)
    n_eff = n

price = disc * m_pay
stderr = disc * std_pay / np.sqrt(n_eff)

print(f'price: {price:.3f}') 
print(f'standard error: {stderr:.3f}')

price: 4.575
standard error: 0.038
